In [ ]:
! pip install -r requirements.txt

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Assistant

In [ ]:
# Assistant stuff

from openai import AzureOpenAI
from typing import Optional
import json
import time

from pathlib import Path


def create_message(
    client: AzureOpenAI,
    thread_id: str,
    role: str = "",
    content: str = "",
    file_ids: Optional[list] = None,
    metadata: Optional[dict] = None,
    message_id: Optional[str] = None,
) -> any:
    """
    Create a message in a thread using the client.

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param role: Message role (user or assistant)
    @param content: Message content
    @param file_ids: Message file IDs
    @param metadata: Message metadata
    @param message_id: Message ID
    @return: Message object

    """
    if metadata is None:
        metadata = {}
    if file_ids is None:
        file_ids = []

    if client is None:
        print("Client parameter is required.")
        return None

    if thread_id is None:
        print("Thread ID is required.")
        return None

    try:
        if message_id is not None:
            return client.beta.threads.messages.retrieve(thread_id=thread_id, message_id=message_id)

        if file_ids is not None and len(file_ids) > 0 and metadata is not None and len(metadata) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, file_ids=file_ids, metadata=metadata
            )

        if file_ids is not None and len(file_ids) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, file_ids=file_ids
            )

        if metadata is not None and len(metadata) > 0:
            return client.beta.threads.messages.create(
                thread_id=thread_id, role=role, content=content, metadata=metadata
            )

        return client.beta.threads.messages.create(thread_id=thread_id, role=role, content=content)

    except Exception as e:
        print(e)
        return None

def poll_run_till_completion(
    client: AzureOpenAI,
    thread_id: str,
    run_id: str,
    available_functions: dict,
    verbose: bool,
    max_steps: int = 10,
    wait: int = 3,
) -> None:
    """
    Poll a run until it is completed or failed or exceeds a certain number of iterations (MAX_STEPS)
    with a preset wait in between polls

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param run_id: Run ID
    @param assistant_id: Assistant ID
    @param verbose: Print verbose output
    @param max_steps: Maximum number of steps to poll
    @param wait: Wait time in seconds between polls

    """

    if (client is None and thread_id is None) or run_id is None:
        print("Client, Thread ID and Run ID are required.")
        return
    try:
        cnt = 0
        while cnt < max_steps:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if verbose:
                print("Poll {}: {}".format(cnt, run.status))
            cnt += 1
            if run.status == "requires_action":
                tool_responses = []
                if (
                    run.required_action.type == "submit_tool_outputs"
                    and run.required_action.submit_tool_outputs.tool_calls is not None
                ):
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls

                    for call in tool_calls:
                        if call.type == "function":
                            if verbose:
                                print(f"Function call: {call.function.name}")
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({"tool_call_id": call.id, "output": tool_response})

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_responses
                )
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "completed":
                break
            time.sleep(wait)

    except Exception as e:
        print(e)

def retrieve_and_print_messages(
    client: AzureOpenAI, thread_id: str, verbose: bool, out_dir: Optional[str] = None
) -> any:
    """
    Retrieve a list of messages in a thread and print it out with the query and response

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param verbose: Print verbose output
    @param out_dir: Output directory to save images
    @return: Messages object

    """

    if client is None and thread_id is None:
        print("Client and Thread ID are required.")
        return None
    try:
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        display_role = {"user": "User query", "assistant": "Assistant response"}

        prev_role = None

        if verbose:
            print("\n\nCONVERSATION:")
        for md in reversed(messages.data):
            if prev_role == "assistant" and md.role == "user" and verbose:
                print("------ \n")

            for mc in md.content:
                # Check if valid text field is present in the mc object
                if mc.type == "text":
                    txt_val = mc.text.value
                # Check if valid image field is present in the mc object
                elif mc.type == "image_file":
                    image_data = client.files.content(mc.image_file.file_id)
                    if out_dir is not None:
                        out_dir_path = Path(out_dir)
                        if out_dir_path.exists():
                            image_path = out_dir_path / (mc.image_file.file_id + ".png")
                            with image_path.open("wb") as f:
                                f.write(image_data.read())

                if verbose:
                    if prev_role == md.role:
                        print(txt_val)
                    else:
                        print("{}:\n{}".format(display_role[md.role], txt_val))
            prev_role = md.role
        return messages
    except Exception as e:
        print(e)
        return None

# Functions

In [ ]:
tools = []
functions_map = {}

In [ ]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import os

import pprint
from azure.devops.v7_0.work_item_tracking.models import JsonPatchOperation

personal_access_token = os.getenv('ADO_PAT')
organization_url = os.getenv('ADO_ORG_URL')
project = os.getenv('ADO_PROJECT')

# Create a connection to the org
credentials = BasicAuthentication('', personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)
wit_client  = connection.clients.get_work_item_tracking_client()

def create_new_workitem(type, title, description, assigned_to = ""):
    work_item_type = type
    # Define the work item fields
    fields = {
        "/fields/System.Title": title,
        "/fields/System.AssignedTo": assigned_to,
        "/fields/System.Description": "test<br/>" +description
    }
    patch_document = [JsonPatchOperation(from_=None, op="add", path=key, value=value) for key, value in fields.items()]
    work_item = wit_client.create_work_item(document=patch_document, project=project, type=work_item_type)

    return f"Created new workitem: {work_item.id}"

create_new_workitem_definition = {
    "type": "function",
    "function": {
        "name": "create_new_workitem",
        "description": "Creates a new work item in Azure DevOps.",
        "parameters": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string",
                    "description": "The type of the work item (e.g., Bug, Task, User Story)",
                },
                "title": {
                    "type": "string",
                    "description": "The title of the work item",
                },
                "description": {
                    "type": "string",
                    "description": "The description of the work item",
                },
                "assignedTo": {
                    "type": "string",
                    "description": "The email of the person to whom the work item is assigned",
                }
            },
            "required": ["type", "title", "description" ],
        },
    },
}

tools.append(create_new_workitem_definition)
functions_map["create_new_workitem"] = create_new_workitem

# Assistant

In [ ]:
def get_assistant(message):
    return  {
        'name' : "ado_assistant",
        'instructions' : """You are an assistant designed to help developer teams manage work items on Azure DevOps.

You have access to the Azure DevOps API and can create, update, and retrieve work items for the user. You can also provide information about the status of work items, assign work items to team members, and help with other aspects of work item management.
""",
        'message': {
            'role': 'user',
            'content': message
        }
    }

In [ ]:
gpt4_azure_openai_endpoint = os.getenv("GPT4_AZURE_OPENAI_ENDPOINT")
gpt4_azure_openai_key = os.getenv("GPT4_AZURE_OPENAI_KEY")
gpt4_azure_openai_api_version = os.getenv("GPT4_AZURE_OPENAI_API_VERSION")
gpt4_deployment_name = os.getenv("GPT4_DEPLOYMENT_NAME")
client = AzureOpenAI(api_key=gpt4_azure_openai_key, api_version=gpt4_azure_openai_api_version, azure_endpoint=gpt4_azure_openai_endpoint)


def call_assistant(assistant_def, verbose_output):
    thread = client.beta.threads.create()
    assistant = client.beta.assistants.create(name=assistant_def['name'], description="", instructions=assistant_def['instructions'], tools=tools, model=gpt4_deployment_name)
    create_message(client, thread.id, assistant_def['message']["role"], assistant_def['message']["content"])
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id, instructions=assistant_def['instructions'])
    poll_run_till_completion(client=client, thread_id=thread.id, run_id=run.id, available_functions=functions_map, verbose=verbose_output, max_steps=50)
    return retrieve_and_print_messages(client=client, thread_id=thread.id, verbose=verbose_output)

# Execution

In [ ]:
import pandas as pd

def csv_to_json(file_path):
    # Read the CSV data
    data = pd.read_csv(file_path)
    
    # Convert the data to JSON
    json_data = data.to_json(orient='records')
    
    return json_data

json_data = csv_to_json('C:\\Users\\mikokono\\Downloads\\TODOsReport.csv')
print(json_data)

In [ ]:
json_data = csv_to_json('C:\\Users\\mikokono\\Downloads\\TODOsReport.csv')


assistant_def = get_assistant(
    f"""
    For each item in the json data, create a new user story work item in Azure DevOps.
    Use the App Name as Workitem title.
    Add each of the other fields with a  not null value to the descirption of the work item, please make sure each of those fields are clearly labeled and are in new line.
    Use <br/> as a new line character.    

    json data with the item information is: 
    {json_data}
    """)

verbose_output = False

call_assistant(assistant_def, verbose_output)